In [1]:
from DijkstraSolver import DijkstraSolver, singleAccessibilitySolve
from BasicSolver import BasicSolver
basicSolver = BasicSolver()

walkingDistanceLimit = 700
timeDeltaLimit = 180 * 60
walkingSpeed = 1.4
isRealTime = True

In [2]:
# Calculating OD matrix
startLocation = {
    "stop_lat": 39.963596, 
    "stop_lon": -83.000944
}
timestamp = 1573942335
args = {
    "timestamp": timestamp, 
    "walkingDistanceLimit": walkingDistanceLimit, 
    "timeDeltaLimit": timeDeltaLimit, 
    "walkingSpeed": walkingSpeed, 
    "isRealTime": isRealTime, 
}

solver, results = singleAccessibilitySolve(args, startLocation)


------ Retrospective Timetable Routing Running... ------


------ Retrospective Timetable Routing Finished... ------
------ Scheduled Timetable Routing Running... ------


------ Scheduled Timetable Routing Finished... ------


In [5]:
# Visualizae PPA
from ipyleaflet import Map, basemaps, Circle, WidgetControl, GeoJSON, LayerGroup
from ipywidgets import IntSlider, ColorPicker, jslink

timeBudget = 30
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(39.963596, -83.000944), zoom=11)
slider = IntSlider(description='Time budget:', min=5, max=120, value=timeBudget)
circleList = []
for receivingStopID, ODRecord in results.items():
    radius = int(max(min((timeBudget * 60 - ODRecord["timeSC"]) * walkingSpeed, walkingDistanceLimit), 0))     
    circle = Circle()
    circle.location = (ODRecord["stop_lat"], ODRecord["stop_lon"])
    circle.radius = int(radius)
    circle.color = "black"
    circle.stroke = False
    circle.fill_opacity = 1
    circle.fill_color = "green"
    circleList.append(circle)
    m.add_layer(circle)

def onChange(e):
    timeBudget = int(e["new"])
    for receivingStopIndex in range(len(results.keys())):
        receivingStopID = list(results.keys())[receivingStopIndex]
        ODRecord = results[receivingStopID]
        radius = int(max(min((timeBudget * 60 - ODRecord["timeSC"]) * walkingSpeed, walkingDistanceLimit), 0))
        circleList[receivingStopIndex].radius = radius

slider = IntSlider(min=5, max=120, value=timeBudget, continuous_update=False)
slider.observe(onChange, names='value')

widget_control1 = WidgetControl(widget=slider, position='topright')
m.add_control(widget_control1)

m


Map(center=[39.963596, -83.000944], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…